In [1]:
#coding=utf-8

from rqalpha.api import *
import traceback
from datetime import *
from sqlalchemy import create_engine
import pandas as pd
from rqalpha.utils.logger import user_log


def dependency():
    return []

def market(market=90):
    if market == 83:
        return 'XSHG'
    elif market == 90:
        return 'XSHE'
    else:
        return ""

def compute(startdt,enddt,context):
    '''
    PE
    :param startdt:
    :param enddt:
    :return:
    '''
    #context.config 对应配置的extra部分
    jydbConf = context.config.jydb
    _jyConnStr = "mysql+pymysql://%s:%s@%s:%s/%s"%(jydbConf.user,jydbConf.passwd,jydbConf.host,jydbConf.port,jydbConf.db)
    engine = create_engine(_jyConnStr)
    
    _category = [1, ]
    _sectors = [1, 2, 6]
    _sql = "SELECT p.TradingDay as date,p.TotalMV as value,a.SecuCode as code,a.SecuMarket" \
                   " FROM LC_DIndicesForValuation as p inner join secumain as a "\
                    "on a.innerCode=p.innerCode where a.SecuMarket in (83,90) " \
                   "and a.SecuCategory in (%s) and a.ListedSector in (%s) " \
                   "and a.ListedState!=9 and p.TradingDay between '%s' and '%s' order by p.TradingDay asc" % (
                        ",".join([str(i) for i in _category]), ",".join([str(i) for i in _sectors])
                        ,startdt.strftime('%Y-%m-%d'),
                       enddt.strftime('%Y-%m-%d'))
#     print(_sql)
    _res = pd.read_sql(sql=_sql,con = engine)
#     market = {90:"XSHE",83:"XSHG"}
    _res.code = _res.code + "." + _res.SecuMarket.apply(market)
    _res = _res.drop(['SecuMarket'],axis=1).set_index(['date','code']).unstack(level=-1)
    _res.columns = _res.columns.droplevel(level=0)
    return _res
                    

         when using: `python -m better_exceptions'. Otherwise, only code
         loaded from files will be inspected!


In [8]:
# test
config= {"extra":{"jydb":{"host":"172.18.44.5","port":3306,"user":"liangh","passwd":"huaxun!@#db","db":"jydb"}}
         ,"factor_data_path":"../../factor_data","factor_data_init_date": "2017-01-01"}
from rqalpha.utils import RqAttrDict
conf = RqAttrDict(config)
from rqalpha.mod.rqalpha_mod_alphaStar_factors.factor_context import FactorContext
context = FactorContext(conf)
context.registerDepending(dependency())
res =  compute(datetime(2017,1,1),datetime(2017,2,1),context)
res

SELECT p.TradingDay as date,p.TotalMV as value,a.SecuCode as code,a.SecuMarket FROM LC_DIndicesForValuation as p inner join secumain as a on a.innerCode=p.innerCode where a.SecuMarket in (83,90) and a.SecuCategory in (1) and a.ListedSector in (1,2,6) and a.ListedState!=9 and p.TradingDay between '2017-01-01' and '2017-02-01' order by p.TradingDay asc


code,000001.XSHE,000002.XSHE,000004.XSHE,000005.XSHE,000006.XSHE,000007.XSHE,000008.XSHE,000009.XSHE,000010.XSHE,000011.XSHE,...,603986.XSHG,603987.XSHG,603988.XSHG,603989.XSHG,603990.XSHG,603993.XSHG,603996.XSHG,603997.XSHG,603998.XSHG,603999.XSHG
date,,,,,,,,,,,,,,,,,,,,,
2017-01-03,1.572810e+11,2.015826e+11,3.732764e+09,7.229807e+09,1.305445e+10,6.028196e+09,2.623679e+10,2.248215e+10,6.386668e+09,1.024517e+10,...,1.779700e+10,6.996348e+09,5.692800e+09,1.118400e+10,4.418400e+09,4.857649e+10,6.081039e+09,8.282400e+09,7.763810e+09,8.654400e+09
2017-01-04,1.572810e+11,2.027495e+11,3.753758e+09,7.420343e+09,1.322995e+10,6.113653e+09,2.637725e+10,2.289052e+10,6.419462e+09,1.037198e+10,...,1.779700e+10,6.924850e+09,6.262400e+09,1.122000e+10,4.433600e+09,4.922418e+10,6.186092e+09,8.366400e+09,7.776801e+09,8.789760e+09
2017-01-05,1.574527e+11,2.035274e+11,3.731924e+09,7.378002e+09,1.325695e+10,6.060531e+09,2.643343e+10,2.265410e+10,6.403065e+09,1.026630e+10,...,1.779700e+10,6.704045e+09,6.215200e+09,1.092600e+10,4.461600e+09,4.922418e+10,6.273135e+09,8.337000e+09,7.681539e+09,8.683200e+09
2017-01-06,1.567659e+11,2.007074e+11,3.691615e+09,7.441514e+09,1.351345e+10,5.878068e+09,2.663006e+10,2.258962e+10,6.321080e+09,1.069957e+10,...,1.779700e+10,6.291877e+09,6.059200e+09,1.079100e+10,4.242400e+09,4.883556e+10,6.261129e+09,8.240400e+09,7.607928e+09,8.758080e+09
2017-01-09,1.571093e+11,2.009019e+11,3.611837e+09,7.483855e+09,1.343245e+10,5.794921e+09,2.677052e+10,2.271858e+10,6.378470e+09,1.076298e+10,...,1.779700e+10,6.264539e+09,6.464800e+09,1.084500e+10,4.232800e+09,4.909464e+10,6.270134e+09,8.265600e+09,7.664219e+09,9.060480e+09
2017-01-10,1.571093e+11,2.001240e+11,3.631992e+09,7.420343e+09,1.317595e+10,5.725631e+09,2.660197e+10,2.282604e+10,6.321080e+09,1.026102e+10,...,1.779700e+10,6.216172e+09,6.477600e+09,1.093500e+10,4.252800e+09,4.961279e+10,6.222110e+09,8.253000e+09,7.690199e+09,8.907840e+09
2017-01-11,1.569376e+11,1.983736e+11,3.564810e+09,7.261563e+09,1.306795e+10,5.739489e+09,2.629297e+10,2.218124e+10,6.075123e+09,9.801335e+09,...,1.779700e+10,6.159394e+09,6.530400e+09,1.089300e+10,4.198400e+09,5.142631e+10,6.039018e+09,8.085000e+09,8.131866e+09,8.631360e+09
2017-01-12,1.571093e+11,1.983736e+11,3.531220e+09,7.187465e+09,1.255495e+10,5.684058e+09,2.629297e+10,2.185884e+10,5.960344e+09,9.743214e+09,...,1.779700e+10,6.094204e+09,6.737600e+09,1.090200e+10,4.202400e+09,5.039001e+10,5.999998e+09,8.047200e+09,7.924023e+09,8.645760e+09
2017-01-13,1.572810e+11,2.120847e+11,3.443044e+09,7.071026e+09,1.192046e+10,5.499285e+09,2.587161e+10,2.164390e+10,5.788174e+09,9.040477e+09,...,1.779700e+10,5.780872e+09,6.503200e+09,1.041000e+10,3.970400e+09,5.077862e+10,6.123060e+09,7.644000e+09,7.846082e+09,8.432640e+09


In [ ]:
from rqalpha.mod.rqalpha_mod_alphaStar_factors.factor_data import FactorData
from datetime import datetime
fd = FactorData("market_value",path=config['factor_data_path']
                ,defaultInitDate = datetime.strptime(config['factor_data_init_date'],"%Y-%m-%d").date())
fd.reset()
fd.append(res)

In [4]:
# evaluateFileDemo
from rqalpha.mod.rqalpha_mod_alphaStar_factors import evaluate_file

config = {
  "base": {
    "start_date": "2018-01-01",
    "end_date": "2018-04-01",
  },
}
# import matplotlib.pyplot as plt
import matplotlib
# matplotlib.use('agg')
print(matplotlib.rcParams['backend'])
factor_file_path = "./market_value.ipynb"

evaluate_file(factor_file_path=factor_file_path,config=config,config_file = "../config_factor.yml")

module://ipykernel.pylab.backend_inline


2018-01-02 INFO factor:market_value compute,startdt:2018-01-02 00:00:00,enddt:2018-03-30


KeyboardInterrupt: 